In [ ]:
import pandas as pd
import platform
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# test comment!
# Detect operating system and initialize the correct WebDriver
if platform.system() == "Windows":
    from selenium.webdriver.edge.service import Service as EdgeService
    # Specify the path to the Edge WebDriver executable
    edge_driver_path = 'C:\\Users\\FreyaSalsbury\\Downloads\\edgedriver_win64\\msedgedriver.exe'
    service = EdgeService(edge_driver_path)
    # Initialize the WebDriver for Microsoft Edge
    driver = webdriver.Edge(service=service)
elif platform.system() == "Darwin":  # Mac
    # Safari driver is built-in on macOS and requires "Allow Remote Automation" enabled in Safari > Develop menu
    driver = webdriver.Safari()
else:
    raise Exception("Unsupported OS")

# Read the CSV file and extract parcel numbers
# This file will have to be updated periodically, as it does not sync to obtain new parcels
# There are 633,340 parcels in the file and 2,130 sq miles in King County, so about 297 parcels per sq mile
# Thus, there is approximately one parcel every 0.0034 sq miles, or 2.2 acres
csv_file_path = 'C:\\Users\\FreyaSalsbury\\King County Data Scraper\\Parcels_for_King_County_with_Address_with_Property_Information___parcel_address_area.csv'
df = pd.read_csv(csv_file_path)
parcel_numbers = df['PIN'].tolist()

# Navigate to King County Parcel Viewer
driver.get("https://gismaps.kingcounty.gov/parcelviewer2/")

# Wait for the page to load
wait = WebDriverWait(driver, 10)

# Change the search type to "Parcel Number"
search_type_dropdown = wait.until(EC.presence_of_element_located((By.ID, "searchTypeDropdown")))
search_type_dropdown.click()
parcel_viewer_option = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value='Parcel Viewer']")))
parcel_viewer_option.click()

# Loop through each parcel number and perform the search
for parcel_number in parcel_numbers[:1]:  # Limit to first 1 for testing; remove [:1] for all parcels
    try:
        # Find the search box and enter the parcel number
        search_box = wait.until(EC.presence_of_element_located((By.ID, "searchBox")))
        search_box.clear()
        search_box.send_keys(parcel_number)
        search_box.submit()

        # Wait for the search results and navigate to the assessor's report
        assessor_report_link = wait.until(EC.presence_of_element_located((By.LINK_TEXT, "Assessor's Report")))
        assessor_report_link.click()

        # Extract property details and sales history
        property_detail_link = wait.until(EC.presence_of_element_located((By.LINK_TEXT, "Property Detail")))
        property_detail_link.click()

        # Scroll down to sales history and extract data
        sales_history = wait.until(EC.presence_of_element_located((By.ID, "salesHistory")))
        sales_data = sales_history.text

        print(f"Sales data for parcel {parcel_number}:")
        print(sales_data)

        # Navigate back to the main search page
        driver.back()
        driver.back()
    except Exception as e:
        print(f"Error processing parcel {parcel_number}: {e}")

# Close the WebDriver
driver.quit()
